In [1]:
# Load libraries
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Create values Selenium
USERNAME = "chosson.jeanfranco@gmail.com" 
PASSWORD =  "@Billyturner@b1o0b1@"
linkedin = 'https://www.linkedin.com'
# Open Selenium
browser = webdriver.Chrome()
browser.get(linkedin)
time.sleep(3)
# Identify email and password inputs and enter in information
email = browser.find_element_by_name('session_key')
password = browser.find_element_by_name('session_password')
email.send_keys(USERNAME + Keys.RETURN)
password.send_keys(PASSWORD + Keys.RETURN)
# Go to profile page and scroll to the bottom of the page to load elements of the page
time.sleep(3)

In [8]:
r = pd.read_csv("result.csv")



In [5]:
r

,name,title,location,profile
0,Joseph Melior,CEO at CHOCOMAX-HAITI,Haiti,https://www.linkedin.com/in/joseph-melior-3a4a...
1,Alexandre Michel,PDG chez Tras-Haïti,Haiti,https://www.linkedin.com/in/alexandre-michel-0...
2,Nick Anderson Azémar,Co-founder chez Blender Haiti,Haiti,https://www.linkedin.com/in/nick-anderson-az%C...
3,Biento Jacques,Co-Founder at GBI Haiti,Haiti,https://www.linkedin.com/in/biento-jacques-871...
4,Valery Fils-Aime,Co-fondateur et Resp. de partenariats ...,Haiti,https://www.linkedin.com/in/valery-fils-aime-6...
...,...,...,...,...
1120,Daniel Dorilas,SEO Specialist / IT Support Specialist...,Haiti,https://www.linkedin.com/in/dexavier/
1121,Jhon kerby Gerlin,Protection Monitor at world vision,Haiti,https://www.linkedin.com/in/jhon-kerby-gerlin-...
1122,Ernest S Sherman,--,Haiti,https://www.linkedin.com/in/ernest-s-sherman-2...
1123,Markenley Belly Saint-Hilaire,Procurement Officer at American Red Cr...,Haiti,https://www.linkedin.com/in/markenley-belly-sa...


In [6]:
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 4 columns):
name        1125 non-null object
title       1124 non-null object
location    1125 non-null object
profile     1125 non-null object
dtypes: object(4)
memory usage: 35.3+ KB


In [9]:

#create empty data frame
Exp_df = pd.DataFrame(columns = ['profile', 'exp_title', 'exp_company', 'exp_dates'])
Edu_df = pd.DataFrame(columns = ['profile', 'ed_name', 'ed_deg', 'ed_dates'])
Ski_df = pd.DataFrame(columns = ['profile', 'skill'])

#Create big loop
#for link in r.loc[0:5,'profile']:
for link in r.loc[:,'profile']:
    if link == 'https://www.linkedin.com#':     #if it equal link then skip
        continue 
    time.sleep(2)
# This section is where you put in the profile link (loaded from the csv file) and browse to it
    search = link
    browser.get(search)
    time.sleep(2)
    browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(.75)
    browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(.75)
    browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(.75)
    browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(.75)
    browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    #browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #raw = urlopen(link).read()
    #page = BeautifulSoup(raw, "html.parser")
    page = BeautifulSoup(browser.page_source, 'lxml')
    
#Experience Section  
    titles = page.find_all('div', class_ = "pv-entity__position-group-pager")
    companies = page.find_all('span', class_ = "pv-entity__secondary-title")
    dates = page.find_all('h4', class_ = "pv-entity__date-range")

    #Put scraped data into exp_df

    arraylen1 = len(page.find_all('div', class_ = "pv-entity__position-group-pager"))

    profile = link
    exp_titles = list(map(lambda x: x.h3.text.strip(), titles))[0:arraylen1]
    exp_companies = list(map(lambda x: x.text.strip(), companies))[0:arraylen1]
    exp_dates = list(map(lambda x: x.text.strip().split('\n')[-1], dates))[0:arraylen1]
    
 #Education Section 
    institution = page.find_all('div', class_ = "pv-entity__degree-info")
    degree = page.find_all('p', class_ = "pv-entity__degree-name")
    dates = page.find_all('p', class_ = "pv-entity__dates")
 
    #Put scraped data into edu_df
    
    arraylen2 = len(page.find_all('div', class_ = "pv-entity__degree-info"))

    profile = link
    ed_name = list(map(lambda x: x.text.strip().split('\n')[-1], institution))[0:arraylen2]
    ed_deg = list(map(lambda x: x.text.strip().split('\n')[-1], degree))[0:arraylen2]
    ed_dates = list(map(lambda x: x.text.strip().split('\n')[-1], dates))[0:arraylen2]
    if len(ed_dates) < arraylen2:
        ed_dates = 'NA'
 #Skill Section 
    skill = page.find_all('span', class_ = "pv-skill-category-entity__name-text")
    
    #Put scraped data into a ski_df
    
    arraylen3 = len(page.find_all('span', class_ = "pv-skill-category-entity__name-text"))
        
    profile = link
    skill = list(map(lambda x: x.text.strip(), skill))[0:arraylen3]
    try:
        temp1 = pd.DataFrame({'profile':profile, 'exp_title':exp_titles, 'exp_company':exp_companies, 'exp_dates':exp_dates})
        temp2 = pd.DataFrame({'profile':profile, 'ed_name':ed_name, 'ed_deg':ed_deg, 'ed_dates':ed_dates}) 
        temp3 = pd.DataFrame({'profile':profile, 'skill':skill})
        Exp_df = Exp_df.append(temp1)
        Edu_df = Edu_df.append(temp2)
        Ski_df = Ski_df.append(temp3)
        print(link, 'completed')
    except:
        print(link, 'skipped')
        continue

https://www.linkedin.com/in/joseph-melior-3a4a7998/ completed
https://www.linkedin.com/in/alexandre-michel-08981b135/ completed
https://www.linkedin.com/in/nick-anderson-az%C3%A9mar-aa2b93145/ skipped
https://www.linkedin.com/in/biento-jacques-871187176/ completed
https://www.linkedin.com/in/valery-fils-aime-69139731/ completed
https://www.linkedin.com/in/mboute/ completed
https://www.linkedin.com/in/maxglyron/ completed
https://www.linkedin.com/in/slenky-rameau-47991090/ completed
https://www.linkedin.com/in/neguerrewidmaer/ skipped
https://www.linkedin.com/in/alfred-alain-b7948293/ completed
https://www.linkedin.com/in/josuestjean/ completed
https://www.linkedin.com/in/cinthia-eliassaint-20435a107/ skipped
https://www.linkedin.com/in/raymond-figaro-excellent-531a4320/ completed
https://www.linkedin.com/in/lunise-jules-8056a91b/ skipped
https://www.linkedin.com/in/myriam-narcisse-458a0a34/ completed
https://www.linkedin.com/in/isabelle-cl%C3%A9ri%C3%A9-a7474434/ completed
https://www.

https://www.linkedin.com/in/stravensky-tilus-854746172/ completed
https://www.linkedin.com/in/lexinor-etienne-02a3a6128/ completed
https://www.linkedin.com/in/simon-nestor-749907182/ completed
https://www.linkedin.com/in/charma-gloria-castor-252693110/ skipped
https://www.linkedin.com/in/carl-henri-proph%C3%A8te-38aab817/ completed
https://www.linkedin.com/in/sofihdesht/ completed
https://www.linkedin.com/in/reginal-r%C3%A9becca-fran%C3%A7ois-49b961163/ completed
https://www.linkedin.com/in/elan-moncher-9b561962/ skipped
https://www.linkedin.com/in/michella-louis-11b43064/ skipped
https://www.linkedin.com/in/laurapincushartman/ skipped
https://www.linkedin.com/in/gvoht-ngo-885186139/ completed
https://www.linkedin.com/in/patrick-jason-bb787696/ completed
https://www.linkedin.com/in/amiral-best%F0%9F%87%AD%F0%9F%87%B9-a3039b198/ completed
https://www.linkedin.com/in/jb-baudelaire-dubic-27543743/ completed
https://www.linkedin.com/in/steven-yonel-junior-almozard-997224137/ skipped
https:

https://www.linkedin.com/in/bernardcherelus/ skipped
https://www.linkedin.com/in/jdessources/ skipped
https://www.linkedin.com/in/daphnee-milsaint-rene-461b2976/ skipped
https://www.linkedin.com/in/david-laforest-52338790/ completed
https://www.linkedin.com/in/predny-alcide-84618813b/ completed
https://www.linkedin.com/in/ritchelle-passius-a335a670/ skipped
https://www.linkedin.com/in/jon-p-charlot-3451b5106/ skipped
https://www.linkedin.com/in/samantha-l-822921117/ completed
https://www.linkedin.com/in/erick-junior-francois-481409142/ completed
https://www.linkedin.com/in/christiantribie/ skipped
https://www.linkedin.com/in/jacquesmarie/ completed
https://www.linkedin.com/in/thbungener/ completed
https://www.linkedin.com/in/jean-cassis-manuel-49381a12a/ completed
https://www.linkedin.com/in/marie-marthe-alix-pierre-louis-regis-77b36937/ completed
https://www.linkedin.com/in/charles-jude-741047161/ skipped
https://www.linkedin.com/in/luckenson-louis-90089442/ completed
https://www.link

https://www.linkedin.com/in/jonathan-jeanty-77302179/ skipped
https://www.linkedin.com/in/chamblain-paul-max-75530062/ completed
https://www.linkedin.com/in/reynald-st-louis-793635150/ completed
https://www.linkedin.com/in/wadler-w-louis-s%C3%A9jour-77378697/ completed
https://www.linkedin.com/in/jean-claude-pierre-b0b261107/ completed
https://www.linkedin.com/in/kendy-pressoir-814179ab/ completed
https://www.linkedin.com/in/rosemie-joly/ completed
https://www.linkedin.com/in/cassamajor-brisly-66469390/ completed
https://www.linkedin.com/in/ernson-augustin-469859138/ completed
https://www.linkedin.com/in/patricia-joseph-495434140/ completed
https://www.linkedin.com/in/monsieur-fresny-4150b4104/ completed
https://www.linkedin.com/in/nestor-lariviere-b8a4383b/ skipped
https://www.linkedin.com/in/samuel-h-jean-philippe-b540b0179/ skipped
https://www.linkedin.com/in/jerry-decime-78b555135/ skipped
https://www.linkedin.com/in/rodeline-auguste-2b0592b0/ skipped
https://www.linkedin.com/in/co

https://www.linkedin.com/in/rubben/ skipped
https://www.linkedin.com/in/chrystinesenecal/ skipped
https://www.linkedin.com/in/hyppolite-mackendy-6b96636b/ completed
https://www.linkedin.com/in/marielle-lapres-92185273/ skipped
https://www.linkedin.com/in/kenny-robert-philippe-auguste-6b358260/ completed
https://www.linkedin.com/in/sandrine-st-plice-89946678/ skipped
https://www.linkedin.com/in/jean-donald-antoine-646076152/ completed
https://www.linkedin.com/in/simonis-christa-49909511a/ skipped
https://www.linkedin.com/in/johanne-w-georges-59b7b511b/ completed
https://www.linkedin.com/in/kerline-rockerline-rock-7b056a49/ completed
https://www.linkedin.com/in/pierre-lyne-desgrottes-5a8658101/ skipped
https://www.linkedin.com/in/jonathan-joseph-81927212a/ skipped
https://www.linkedin.com/in/vickymc21/ skipped
https://www.linkedin.com/in/laurie-destramy-94b123128/ skipped
https://www.linkedin.com/in/gessika-thomas-385aa313a/ completed
https://www.linkedin.com/in/lynn-stacy-sanon-53b24289

https://www.linkedin.com/in/weedens-videau-60764162/ completed
https://www.linkedin.com/in/sadais-jeannite-48674843/ completed
https://www.linkedin.com/in/indy-edouard-b34658117/ skipped
https://www.linkedin.com/in/strauss-vedrine-b27aba59/ skipped
https://www.linkedin.com/in/patrice-djen-s-mervil-e-021346150/ skipped
https://www.linkedin.com/in/romelus-ismael-40b4a55a/ completed
https://www.linkedin.com/in/vanessa-ariel-paul-2012b977/ skipped
https://www.linkedin.com/in/saintil-smyrne-00292b44/ skipped
https://www.linkedin.com/in/marlie-landy-joseph-864215149/ skipped
https://www.linkedin.com/in/yana-perpetue-vilton-49495826/ completed
https://www.linkedin.com/in/macennat-jeanwilly-642029174/ skipped
https://www.linkedin.com/in/yamilee-payen-76150944/ skipped
https://www.linkedin.com/in/franoshka-telemaque-b9bb9bb0/ completed
https://www.linkedin.com/in/celinord-rickens-92b4b8197/ completed
https://www.linkedin.com/in/jodna-lafond-0036a6b1/ completed
https://www.linkedin.com/in/meliss

https://www.linkedin.com/in/jeff-lundi-5808a6122/ skipped
https://www.linkedin.com/in/medialine-bernard-67087518b/ completed
https://www.linkedin.com/in/saitha-salomon-95763a196/ completed
https://www.linkedin.com/in/in%C3%A8s-jolibois/ completed
https://www.linkedin.com/in/jacques-sedric-antoine/ skipped
https://www.linkedin.com/in/ingrid-mirlanda-delva-229b2831/ completed
https://www.linkedin.com/in/james-petit-b8464aaa/ skipped
https://www.linkedin.com/in/gucci-cheristin-629b57132/ skipped
https://www.linkedin.com/in/erold-ezerbe-a0218072/ completed
https://www.linkedin.com/in/franscia-c%C3%A9lestin-1a5b03151/ completed
https://www.linkedin.com/in/mirl%C3%A8ne-verdier-14117980/ skipped
https://www.linkedin.com/in/wodney-jeudy-13b5651a3/ completed
https://www.linkedin.com/in/maguedala-selmour-832bb745/ completed
https://www.linkedin.com/in/islande-l%C3%A9andre-43927056/ skipped
https://www.linkedin.com/in/nehyssa-julien-b88a3011b/ completed
https://www.linkedin.com/in/huny-laguerre-4

WebDriverException: Message: chrome not reachable
  (Session info: chrome=84.0.4147.89)


In [10]:
Exp_df.shape

(0, 4)

In [11]:
Edu_df

,profile,ed_name,ed_deg,ed_dates
0,https://www.linkedin.com/in/alexandre-michel-0...,Sciences de la Communication et des relations ...,Licence,2014 – 2018
0,https://www.linkedin.com/in/biento-jacques-871...,License,Diplome,2016 – 2017
1,https://www.linkedin.com/in/biento-jacques-871...,License,Bachelor's degree,2014 – 2015
0,https://www.linkedin.com/in/valery-fils-aime-6...,Project Management,Msc Project Managment,NA
1,https://www.linkedin.com/in/valery-fils-aime-6...,Gestion des PME,Gestion des Affaires,NA
...,...,...,...,...
1,https://www.linkedin.com/in/jothsaina-pierre-a...,Law,licentiate degree,2014 – 2018
2,https://www.linkedin.com/in/jothsaina-pierre-a...,"Langues(anglais, espagnol)",diplome de fin d'année,2007 – 2014
0,https://www.linkedin.com/in/rodrigue-son-s-mal...,Droit,Science juridique,2017 – 2020
0,https://www.linkedin.com/in/stev-amabl-a31595124/,"Administration et gestion des affaires, général",Bachelor of Business Administration - BBA,NA


In [12]:
Ski_df

,profile,skill
0,https://www.linkedin.com/in/alexandre-michel-0...,Public Speaking
1,https://www.linkedin.com/in/alexandre-michel-0...,Social Media
2,https://www.linkedin.com/in/alexandre-michel-0...,Microsoft Office
0,https://www.linkedin.com/in/biento-jacques-871...,Graphic Design
1,https://www.linkedin.com/in/biento-jacques-871...,Video Production
...,...,...
1,https://www.linkedin.com/in/charles-barbara-30...,Customer Service
2,https://www.linkedin.com/in/charles-barbara-30...,Banking
0,https://www.linkedin.com/in/jothsaina-pierre-a...,Microsoft Office
1,https://www.linkedin.com/in/jothsaina-pierre-a...,Microsoft Word
